In [24]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

In [56]:
# 인증단계를 처리하는 API
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json'
params = {'consumer_key':'196df7f66e8e4b78a99f','consumer_secret':'28fbc09c175748d8a935'}
res = requests.get(url, params=params).text
accesstoken = json.loads(res)['result']['accessToken']
accesstoken

'0760ffd8-3889-401e-829f-37a236890165'

In [182]:
# 좌표변환을 제공
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json'
params = {'accessToken':accesstoken, 'src':5179, 'dst':4326, 'posX':960652.321774961, 'posY':1945259.08378017}
res = requests.get(url, params=params).text
res = json.loads(res)['result']
res

{'posY': 37.50576594071522, 'posX': 127.05480746860431}

In [181]:
# 지오코딩 : 입력된 주소에 대한 위치를 제공
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json'
params = {'accessToken':accesstoken, 'address':'테헤란로 432'}
res = requests.get(url, params=params).text
res = json.loads(res)['result']['resultdata'][0]
res

{'ri_nm': 'null',
 'road_nm_sub_no': '0',
 'sgg_cd': '230',
 'adm_cd': '1123063',
 'road_nm_main_no': '432',
 'leg_cd': '11680106',
 'road_nm': '테헤란로',
 'bd_sub_nm': 'null',
 'addr_type': '6',
 'sido_nm': '서울특별시',
 'sgg_nm': '강남구',
 'sido_cd': '11',
 'road_cd': '3122010',
 'bd_main_nm': '동부금융센터',
 'adm_nm': '대치4동',
 'jibun_main_no': '891',
 'jibun_sub_no': '10',
 'ri_cd': '1168010600',
 'y': '1945259.08378017',
 'leg_nm': '대치동',
 'x': '960652.321774961'}

In [177]:
# 좌표를 이용한 집계구, 행정동코드 정보 조회
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/personal/findcodeinsmallarea.json'
params = {'accessToken':accesstoken, 'x_coor':x, 'y_coor':y}
res = requests.get(url, params=params).text
res = json.loads(res)['result']
res

{'sgg_cd': '230',
 'emdong_cd': '63',
 'sido_nm': '서울특별시',
 'sgg_nm': '강남구',
 'emdong_nm': '대치4동',
 'tot_reg_cd': '1123063010002',
 'sido_cd': '11'}

In [142]:
# 임의 영역내 전개도 건물 정보를 제공
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/figure/buildingarea.json'
params = {'accessToken':accesstoken, 'minx':x, 'miny':y, 'maxx':x, 'maxy':y}
res = requests.get(url, params=params).text
res = json.loads(res)['result'][0]
sufid = res['sufid']
highest_flr = res['highest_flr']
lowest_flr = res['lowest_flr']
sufid, lowest_flr, highest_flr

('200709606523471945262850', '-7', '35')

In [143]:
# 층별 시설물 공간 속성 정보를 제공
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/figure/flooretcfacility.json'
params = {'accessToken':accesstoken, 'sufid':sufid, 'flr_no':11}
res = requests.get(url, params=params).text
res = json.loads(res)
res

{'id': 'API_0805',
 'errMsg': '검색결과가 존재하지 않습니다.',
 'errCd': -100,
 'trId': 'L7GJ_API_0805_1532227621016'}

In [141]:
# 건물층별 사업체 정보
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/figure/floorcompanyinfo.json'
params = {'accessToken':accesstoken, 'sufid':sufid, 'flr_no':11}
res = requests.get(url, params=params).text
res = json.loads(res)['result']
res

{'theme_cd_list': [{'theme_cd_nm': '기업(기타금융업)', 'theme_cd': '8006'}],
 'companylist': [{'center_y': 1945261.940034093,
   'center_x': 960627.3591458419,
   'decilist_serial': '0000354087',
   'theme_cd': '8006',
   'corp_nm': '동부화재해상보험(주)'},
  {'center_y': 1945271.9602075445,
   'center_x': 960660.4779043624,
   'decilist_serial': '0002357660',
   'theme_cd': '8006',
   'corp_nm': '(주)동부화재근로복지기금'}],
 'facilitylist': []}

In [184]:
# 행정구역 및 도로명 주소에 대해서 단계별로 조회
url = 'https://sgisapi.kostat.go.kr/OpenAPI3/addr/stage.json'
params = {'accessToken':accesstoken}
res = requests.get(url, params=params).text
res = json.loads(res)['result']
addr = [(r['addr_name'], r['x_coor'], r['y_coor']) for r in res]
addr

[('서울특별시', '953932', '1952053'),
 ('부산광역시', '1141856', '1690474'),
 ('대구광역시', '1099471', '1764433'),
 ('인천광역시', '929709', '1939946'),
 ('광주광역시', '940934', '1685175'),
 ('대전광역시', '989651', '1817013'),
 ('울산광역시', '1164223', '1728579'),
 ('세종특별자치시', '981426', '1843889'),
 ('경기도', '956444', '1919648'),
 ('강원도', '1020213', '1987302'),
 ('충청북도', '999288', '1848669'),
 ('충청남도', '926074', '1851555'),
 ('전라북도', '964651', '1758283'),
 ('전라남도', '905137', '1647339'),
 ('경상북도', '1112095', '1817430'),
 ('경상남도', '1108455', '1694249'),
 ('제주특별자치도', '906937', '1500136')]